In [38]:
import pandas as pd
import numpy as np

# Load the events dataset
data = pd.read_csv('C:/Users/nafla/Downloads/article/dataset/filtered_sessions.csv')
data.head()

,timestamp,visitorid,itemid,session_id,global_session_id,time_since_session_start,addtocart,transaction,view
0,2015-09-11 20:49:49.439,0,285930,0,0_0,0.000,False,False,True
1,2015-09-11 20:52:39.591,0,357564,0,0_0,170.152,False,False,True
2,2015-09-11 20:55:17.175,0,67045,0,0_0,327.736,False,False,True
3,2015-08-30 06:39:38.318,6,253615,1,6_1,0.000,False,False,True
4,2015-08-30 06:40:23.805,6,344723,1,6_1,45.487,False,False,True


In [39]:
# finding unique users
unique_users = data['visitorid'].unique()

# Set a random seed for reproducibility
np.random.seed(42)

# Calculate the number of users to sample
sample_size = int(len(unique_users) * 0.20)

# Randomly sample user IDs
sampled_users = np.random.choice(unique_users, size=sample_size, replace=False)

In [40]:
# Filter the original data to keep only the sessions of sampled users
sampled_data = data[data['visitorid'].isin(sampled_users)]

# Optional: Verify the results
sampled_data.head()
print(f"Number of sessions in the sampled data: {sampled_data['global_session_id'].nunique()}")
print(f"Number of users in the sampled data: {sampled_data['visitorid'].nunique()}")


Number of sessions in the sampled data: 64120
Number of users in the sampled data: 55642


In [41]:
sampled_data.head()

,timestamp,visitorid,itemid,session_id,global_session_id,time_since_session_start,addtocart,transaction,view
15,2015-07-13 16:29:23.786,36,176286,0,36_0,0.000,False,False,True
16,2015-07-13 16:32:48.853,36,176286,0,36_0,205.067,False,False,True
29,2015-06-22 15:38:37.948,65,52383,0,65_0,0.000,False,False,True
30,2015-06-22 15:39:17.183,65,168656,0,65_0,39.235,False,False,True
31,2015-06-22 15:39:23.298,65,52383,0,65_0,45.350,False,False,True


In [42]:
from sklearn.preprocessing import MinMaxScaler

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Assuming 'sampled_data' is your DataFrame from the previous step
sampled_data['time_since_session_start'] = scaler.fit_transform(sampled_data[['time_since_session_start']])


C:\Users\nafla\AppData\Local\Temp\ipykernel_13568\2524327454.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled_data['time_since_session_start'] = scaler.fit_transform(sampled_data[['time_since_session_start']])


In [43]:
from sklearn.preprocessing import LabelEncoder

# Initialize the Label Encoder
encoder = LabelEncoder()

# Encode the itemid
sampled_data['itemid'] = encoder.fit_transform(sampled_data['itemid'])


C:\Users\nafla\AppData\Local\Temp\ipykernel_13568\214686607.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled_data['itemid'] = encoder.fit_transform(sampled_data['itemid'])


In [44]:
# Assuming 'sampled_data' is your DataFrame
# First, ensure your data is sorted by session and timestamp
sampled_data.sort_values(['global_session_id', 'timestamp'], inplace=True)

# Add an interaction index within each session
sampled_data['interaction_index'] = sampled_data.groupby('global_session_id').cumcount()

# Now, you can drop the original 'timestamp' column if you decide it's not needed for the initial model
# sampled_data.drop(columns=['timestamp'], inplace=True)

# Your data is now ready with a simple sequence identifier for each session
print(sampled_data.head())

                      timestamp  visitorid  itemid  session_id  \
555248  2015-09-07 17:14:59.325    1000030    1750           0   
555249  2015-09-07 17:15:58.628    1000030    1750           0   
555254  2015-08-31 05:03:10.068    1000039   14394           0   
555255  2015-08-31 05:04:33.631    1000039   26979           0   
555256  2015-08-31 05:07:44.643    1000039   47198           0   

       global_session_id  time_since_session_start  addtocart  transaction  \
555248         1000030_0                  0.000000      False        False   
555249         1000030_0                  0.011976      False         True   
555254         1000039_0                  0.000000      False        False   
555255         1000039_0                  0.016876      False        False   
555256         1000039_0                  0.055451      False        False   

         view  interaction_index  
555248   True                  0  
555249  False                  1  
555254   True                

C:\Users\nafla\AppData\Local\Temp\ipykernel_13568\2752901800.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled_data.sort_values(['global_session_id', 'timestamp'], inplace=True)
C:\Users\nafla\AppData\Local\Temp\ipykernel_13568\2752901800.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled_data['interaction_index'] = sampled_data.groupby('global_session_id').cumcount()


In [45]:
# Group by 'global_session_id' and aggregate columns
grouped = sampled_data.groupby('global_session_id').agg(list)

# Checking the grouped data structure
print(grouped.head())


                                                           timestamp  \
global_session_id                                                      
1000030_0          [2015-09-07 17:14:59.325, 2015-09-07 17:15:58....   
1000039_0          [2015-08-31 05:03:10.068, 2015-08-31 05:04:33....   
1000049_0          [2015-07-22 16:15:47.678, 2015-07-22 16:24:17....   
100013_0           [2015-07-17 18:52:53.392, 2015-07-17 18:53:27....   
1000145_0          [2015-05-08 21:41:57.620, 2015-05-08 21:52:28....   

                                              visitorid  \
global_session_id                                         
1000030_0                            [1000030, 1000030]   
1000039_0          [1000039, 1000039, 1000039, 1000039]   
1000049_0                            [1000049, 1000049]   
100013_0                               [100013, 100013]   
1000145_0                            [1000145, 1000145]   

                                         itemid    session_id  \
global_session_i

In [46]:
specific_user_session = sampled_data[(sampled_data['visitorid'] == 36) & (data['global_session_id'] == '36_0')]

# Display the filtered data
print(specific_user_session)

                  timestamp  visitorid  itemid  session_id global_session_id  \
15  2015-07-13 16:29:23.786         36   18147           0              36_0   
16  2015-07-13 16:32:48.853         36   18147           0              36_0   

    time_since_session_start  addtocart  transaction  view  interaction_index  
15                  0.000000      False        False  True                  0  
16                  0.041414      False        False  True                  1  


C:\Users\nafla\AppData\Local\Temp\ipykernel_13568\286005686.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  specific_user_session = sampled_data[(sampled_data['visitorid'] == 36) & (data['global_session_id'] == '36_0')]


In [47]:
# Assuming 'sampled_data' is already loaded and contains a 'timestamp' column
sampled_data.sort_values('timestamp', inplace=True)

# Calculate indices for splitting
total_count = len(sampled_data)
train_end = int(total_count * 0.70)
validation_end = int(total_count * 0.85)

# Split the data
train_data = sampled_data[:train_end]
validation_data = sampled_data[train_end:validation_end]
test_data = sampled_data[validation_end:]

C:\Users\nafla\AppData\Local\Temp\ipykernel_13568\3574592186.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled_data.sort_values('timestamp', inplace=True)


In [48]:
# Verify the minimum sequence length in validation and test sets
min_validation_length = validation_data.groupby('global_session_id').size().min()
min_test_length = test_data.groupby('global_session_id').size().min()

print(f"Minimum interactions in validation sequences: {min_validation_length}")
print(f"Minimum interactions in test sequences: {min_test_length}")


Minimum interactions in validation sequences: 1
Minimum interactions in test sequences: 1


In [49]:
train_data.head()

,timestamp,visitorid,itemid,session_id,global_session_id,time_since_session_start,addtocart,transaction,view,interaction_index
384073,2015-05-03 03:00:04.384,693516,30968,0,693516_0,0.000000,True,False,False,0
384074,2015-05-03 03:00:26.228,693516,30968,0,693516_0,0.004411,False,False,True,1
384075,2015-05-03 03:00:29.427,693516,30968,0,693516_0,0.005057,True,False,False,2
144845,2015-05-03 03:01:07.992,260113,13013,0,260113_0,0.000000,False,False,True,0
144846,2015-05-03 03:01:50.079,260113,5355,0,260113_0,0.008500,False,False,True,1
